In [1]:
import os
import csv

import pandas as pd
import json

from utils_part2 import clean_csv, Node

## Configurations

In [2]:
input_file_path = os.path.join('.','input_data','algorithms part dataset.csv')
clean_file = True

In [3]:
# Run unit tests
!pytest

============================= test session starts =============================
platform win32 -- Python 3.9.12, pytest-7.1.1, pluggy-1.0.0
rootdir: C:\Users\jpicao\git_repos\interview_case_studies\part2
plugins: anyio-3.5.0
collected 2 items

test_utils_part2.py ..                                                   [100%]

============================== 2 passed in 0.54s ==============================


# `Node()` class description

## `Node()` class attributes

- `self.value`: The actual data or value in the instance. In this exercise this attribute will contain the values coming from the provided csv file (except the column names, which define the node type).
- `self.node_type`: Describes the type of information that the instance represents. In this exercise, the node_types will be the different columns in the provided csv file. 
- `self.children`: A list containing the instance children. Children are also instances of type `Node()`.
- `self.count_rows`: An integer counting the number of table rows that are represented by the sub-tree starting in the current node. If `self` is a leaf node, then `self.count_rows` is used to count the number of duplicates (i.e., `self.count_rows > 1` means duplicates were found), otherwise it is used mainly for internal validation of the tree structure.
- `self.node_hierarchy`: Defines the children node hierarchy levels of the sub-tree starting in the current node. For example, in this exercise the root node will have `self.node_hierarchy = ['ID', 'GAME_NAME', 'BEHAVIOUR', 'PLAY_PURCHASE', 'NONE]`. It is probably an unnecessary attribute that over complicates the class, but didn't have time to refactor.

## `Node()` class attributes

# Compare result with pandas

In [4]:
input_file_path

'.\\input_data\\algorithms part dataset.csv'

In [5]:
if clean_file:
    clean_csv(input_file_path)
    input_file_path = input_file_path.replace('.csv', '_cln.csv')

In [6]:
input_file_path

'.\\input_data\\algorithms part dataset_cln.csv'

In [7]:
df_input = pd.read_csv(input_file_path).iloc[:1000,:]

In [8]:
df_input

,ID,GAME_NAME,BEHAVIOUR,PLAY_PURCHASE,NONE
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0
...,...,...,...,...,...
995,97298878,Burnout Paradise The Ultimate Box,play,0.6,0
996,97298878,Mirror's Edge,purchase,1.0,0
997,97298878,Mirror's Edge,play,0.5,0
998,97298878,Crysis 2 Maximum Edition,purchase,1.0,0


In [9]:
try:
    del tree
except NameError:
    pass

tree = Node(df_input.columns.to_list())

In [10]:
tree.add_table(df_input)

In [11]:
df_out = tree.tree_to_df(duplicates ='drop')

In [12]:
df_out

,ID,GAME_NAME,BEHAVIOUR,PLAY_PURCHASE,NONE
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0
...,...,...,...,...,...
995,97298878,Burnout Paradise The Ultimate Box,play,0.6,0
996,97298878,Mirror's Edge,purchase,1.0,0
997,97298878,Mirror's Edge,play,0.5,0
998,97298878,Crysis 2 Maximum Edition,purchase,1.0,0


In [13]:
pd.testing.assert_frame_equal(df_out, df_input.drop_duplicates().reset_index(drop=True))

In [ ]:
row_values_dict = {a:'', }.values()

In [ ]:
row_str = ','.join(str(value) for value in row_values_dict.values()) + '\n'

In [ ]:
[53875128,Sir, You Are Being Hunted,purchase,1.0,0]

In [25]:
','.join(['1', '3', '4'])

'1,3,4'

In [ ]:
del tree
tree = Node(df_easy.columns.to_list())

In [ ]:
tree.add_table(df_easy)

In [ ]:
tree.print_tree()

In [ ]:
tree.print_table(duplicates='ignore')

In [ ]:
tree.tree_to_df(duplicates ='ignore')

In [ ]:
df_3

In [59]:
input_file_path = os.path.join('.','input_data','algorithms part dataset.csv')
clean_csv_4(input_file_path)

In [51]:
line = 'ID,GAME_NAME,BEHAVIOUR,PLAY_PURCHASE,NONE;\r\n'
line = '"151603712,""The Elder Scrolls V Skyrim"",purchase,1.0,0";\r\n'

In [52]:
cln_row = line
cln_row = re.sub(r'^"', '', cln_row)
cln_row = re.sub(r'\r\n$', r'\n', cln_row)
cln_row = re.sub(r'";$', '', cln_row)

In [53]:
cln_row

'151603712,""The Elder Scrolls V Skyrim"",purchase,1.0,0\n'

In [13]:
    with open('ahjdvcj.csv', 'w', newline='') as csvfile:
        # writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer = csv.writer(csvfile, delimiter=',')
        
        for row in final_list_of_rows[:10]:
            writer.writerow(row)

In [ ]:
len(row)

In [22]:
import re

In [44]:
re.sub(r'";\r\n$', r'', '"151603712,""The Elder Scrolls V Skyrim"",purchase,1.0,0";\r\n')

'"151603712,""The Elder Scrolls V Skyrim"",purchase,1.0,0'

In [ ]:
re.sub(r'^"', '', '""BioShoock""')

In [24]:
re.sub(r'^"', '', '""BioShoock""')

'"BioShoock""'

In [ ]:
re.sub(r'"{1}(?=\w*)','','"53875128')

In [ ]:
re.sub(r'"{1}(?=.+^")','','"53875128')

In [ ]:
re.sub(r'"{1}(?=.+)^["]','','"53875128')

In [ ]:
(?=u)

In [ ]:
row[3]

In [6]:

with open(input_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile, quotechar='"', delimiter=',', quoting=csv.QUOTE_NONE)
    
    # Initializations
    final_list_of_rows = []
    i = 0

    for row in reader:
        i += 1
        if i>375 and i<385:
            row_cln = row.copy()
            if len(row_cln) == 0:
                # empty row
                continue
            else:
                final_list_of_rows.append(row_cln[1:-2])       
#         i += 1
        
#         if i>375 and i<385:
#             print(type(row), ': ', row)
    

In [ ]:
final_list_of_rows

In [ ]:
pd.read_csv(input_file_path)

In [3]:
if clean_file:
    clean_csv_3(input_file_path)
    input_file_path = input_file_path.replace('.csv', '_cln.csv')

In [4]:
input_file_path

'.\\input_data\\algorithms part dataset_cln.csv'

In [5]:
df_in = pd.read_csv(input_file_path)

ParserError: Error tokenizing data. C error: Expected 5 fields in line 379, saw 6


In [ ]:
del tree
tree = Node(df_easy.columns.to_list())

In [ ]:
tree.add_table(df_easy)

In [ ]:
tree.print_tree()

In [ ]:
tree.print_table(duplicates='ignore')

In [ ]:
tree.tree_to_df(duplicates ='ignore')

In [ ]:
# import anytree

In [ ]:
clean_csv(os.path.join('.', 'smpl.csv'))

In [ ]:
input_file_path = os.path.join('.', 'smpl_cln.csv')

In [ ]:
type(input_file_path)

In [ ]:
df = pd.read_csv(input_file_path)

In [ ]:
df

In [ ]:
df_in = df[['GAME_NAME']]

In [ ]:
df[['GAME_NAME']].to_dict()

In [ ]:
df_out = pd.DataFrame(
    {'GAME_NAME': {0: 'The Elder Scrolls V Skyrim',
  1: 'The Elder Scrolls V Skyrim',
  2: 'Fallout 4',
  3: 'Fallout 4',
  4: 'Fallout 4',
  5: 'Spore',
  6: 'Spore',
  7: 'Spore',
  8: 'Fallout New Vegas'}}
)

In [ ]:
!pytest

In [ ]:
df_out.equals(df_in)

In [ ]:
pd.testing.assert_frame_equal(df_in, df_out, check_dtype=False)

In [ ]:
df_in

In [ ]:
df_out

In [ ]:
pd.DataFrame({'GAME_NAME': {0: 'The Elder Scrolls V Skyrim',
  1: 'The Elder Scrolls V Skyrim',
  2: 'Fallout 4',
  3: 'Fallout 4',
  4: 'Fallout 4',
  5: 'Spore',
  6: 'Spore',
  7: 'Spore',
  8: 'Fallout New Vegas'}})

## Add table line by line

In [ ]:
df_easy = df[['GAME_NAME', 'BEHAVIOUR', 'ID']]
# df_easy = df[['GAME_NAME']]
df_easy

In [ ]:
tree = Node(df_easy.columns.to_list())

In [ ]:
tree.node_type

In [ ]:
tree.node_hierarchy

In [ ]:
tree.children

In [ ]:
tree.value

In [ ]:
tree.count_rows

In [ ]:
tree.print_tree()

In [ ]:
row = df_easy.iloc[5,:]
row

In [ ]:
for index, row in df_easy.iterrows():
    print(row.to_list(), ' added. \n')

In [ ]:
for index, row in df_easy.iterrows():
    tree.add_row(row)
    print(row.to_list(), ' added. \n')
    tree._validate_row_counts()
    tree.print_tree(ident_count=0)
    print('\n')

In [ ]:
df_easy

In [ ]:
tree.print_table()

In [ ]:
tree.print_table(duplicates='find')

In [ ]:
tree.print_table(duplicates='ignore')

In [ ]:
tree.print_table()

In [ ]:
tree.print_table(duplicates='ignore', to='test.csv')

In [ ]:
type(df)

In [ ]:
isinstance(df, pd.core.frame.DataFrame)

In [ ]:
tree.print_table(duplicates='ignore')

In [ ]:
tree.print_table(duplicates='find')

In [ ]:
!pytest

## At table at once

In [ ]:
df_easy

In [ ]:
del tree
tree = Node(df_easy.columns.to_list())

In [ ]:
tree.add_table(df_easy)

In [ ]:
tree.print_tree()

In [ ]:
tree.print_table(duplicates='ignore')

In [ ]:
tree.tree_to_df(duplicates ='ignore')

In [ ]:
df_3

In [ ]:
class Tree(object):
    "Generic tree node."
    def __init__(self, name='root', children=None):
        self.name = name
        self.children = []
        if children is not None:
            for child in children:
                self.add_child(child)
    def __repr__(self):
        return self.name
    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)